In [8]:
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma 
from langchain_openai import OpenAIEmbeddings
from pathlib import Path
from chromadb.utils import embedding_functions
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr
import re

load_dotenv()

True

In [2]:
vectorstore = Chroma( 
    embedding_function = OpenAIEmbeddings(model="text-embedding-3-large"),
    collection_name="asrin-getirdigi-tereddutler", 
    persist_directory="./.chromadb",
    collection_metadata={"embeeding_model":"text-embedding-3-large",
    "chunk_size":1024,
    "chunk_overlap":100})

In [4]:
def get_context(query: str, vectorstore) ->list:
    results = vectorstore.similarity_search_with_score(
    query, k=3)
    return results

In [13]:

def generate_response(query: str, retrieved_docs):
    llm = ChatOpenAI(model="gpt-4o-mini",temperature=0) 
    prompt = hub.pull("rlm/rag-prompt")
    result = {"query": query, "retrieved_docs": retrieved_docs, "response": None}
    generation_chain = prompt | llm | StrOutputParser()
    result["response"] = generation_chain.invoke({"context": retrieved_docs, "question": query})
    return result

In [6]:
def is_meaningful_input(user_input: str) -> bool:
    """
    Kullanıcıdan gelen inputun anlamlı olup olmadığını kontrol eder.
    
    Args:
        user_input (str): Kullanıcıdan gelen metin.
    
    Returns:
        bool: Anlamlı ise True, değilse False.
    """
    # Boş veya sadece boşluklardan oluşuyorsa False döndür
    if not user_input.strip():
        return False
    
    # Sadece semboller veya rastgele karakterler varsa False döndür
    # Alfanümerik (harf veya rakam) karakterlerin olup olmadığını kontrol eder
    if not re.search(r'[a-zA-Z0-9]', user_input):
        return False
    
    # Özel bir uzunluk kontrolü (ör. en az 3 karakter anlamlıdır)
    if len(user_input.strip()) < 3:
        return False
    
    return True

In [9]:
def create_context(result):
    contexts =""
    for r in result:
        contexts +=r[0].page_content
    return contexts

In [10]:
def get_vectordb():
    vectorstore = Chroma( 
    embedding_function = OpenAIEmbeddings(model="text-embedding-3-large"),
    collection_name="asrin-getirdigi-tereddutler", 
    persist_directory="./.chromadb",
    collection_metadata={"embeeding_model":"text-embedding-3-large",
    "chunk_size":1024,
    "chunk_overlap":100})
    return vectorstore

In [11]:
message = "Allah her şeyi yarattı, -hâşâ- O’nu kim yarattı?"
query2 ="Nürnbergde hava nasil?"

In [48]:
retrieved_docs = []
if is_meaningful_input(message):
    retrieved_docs = get_context(message, get_vectordb())
    if retrieved_docs[0][1]>1:
        print("Sormus oldugunuz soru HocaGPT nin kapsami disinda oldugundan cevap veremiyorum. Lütfen konu ile ilgili bir soru sormayi deneyin.")
    else:
        context = create_context(retrieved_docs)
        gpt_response = generate_response(message, context)
        print(gpt_response['response'])
else:
    print("Lütfen anlamlı bir soru sorunuz.")

Allah, varlığı kendinden olan ve yaratılmamış tek varlıktır. Yaratılan her şey muhtaç ve sonradan var olmuştur. Bu nedenle, "O'nu kim yarattı?" sorusu, yanlış bir anlayışın sonucudur.


In [49]:
import json
# Document türündeki verileri JSON'a dönüştür
def document_to_dict(documents):
    dict_list = []
    for doc in documents:
        doc_dict ={ "id": None, "metadata": None, "page_content": None,"relevance_score": None}
        
        doc_dict['id']=doc[0].id
        doc_dict['metadata']=doc[0].page_content
        doc_dict['page_content']=doc[0].metadata
        doc_dict['relevance_score']=doc[1]
        dict_list.append(doc_dict)
    return dict_list
        
        
    



In [ ]:
from datetime import datetime
history = {"id":None,"retrieved_docs": None, "gpt_response": None, "message": None, "zeitstempel": None}
retrieved_docs = document_to_dict(retrieved_docs)
history['retrieved_docs']= retrieved_docs
history['response']=gpt_response['response']
history['message']=message
history['zeitstempel']=datetime.now().isoformat()
history['id']=1

In [53]:
history.keys()

dict_keys(['retrieved_docs', 'gpt_response', 'message', 'zeitstempel', 'response'])

In [40]:
res = document_to_dict(retrieved_docs)

In [21]:
import json
try:
    with open('chat_histroy.json', 'r') as datei:
        json_data = json.load(datei)
except Exception as errror:
    print(f'errror: {errror}')

In [22]:
data = {
    "ad": "Ahmet",
    "yas": 25,
    "sehir": "İstanbulwer"
}

In [23]:
json_data.append(data)

In [24]:
with open("chat_histroy.json", "w") as json_file:
        json.dump(json_data, json_file, indent=4)
